In [ ]:
import numpy as np
import pandas as pd
import keras
import keras.backend as K
from keras.optimizers import Adam
from keras.models import Sequential
from keras.utils import Sequence
from keras.layers import *
import matplotlib.pyplot as plt

In [100]:
path = "data/sudoku.csv"
data = pd.read_csv(path)
try:
    data = pd.DataFrame({"quizzes":data["puzzle"],"solutions":data["solution"]})
except:
    pass


In [48]:
#Utility Functions
class DataGenerator(Sequence):
    def __init__(self, df,batch_size = 16,subset = "train",shuffle = False, info={}):
        super().__init__()
        self.df = df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.subset = subset
        self.info = info
        
        self.data_path = path
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.df)/self.batch_size))
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle==True:
            np.random.shuffle(self.indexes)
            
    def __getitem__(self,index):
        X = np.empty((self.batch_size, 9,9,1))
        y = np.empty((self.batch_size,81,1))
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        for i,f in enumerate(self.df['quizzes'].iloc[indexes]):
            self.info[index*self.batch_size+i]=f
            X[i,] = (np.array(list(map(int,list(f)))).reshape((9,9,1))/9)-0.5
        if self.subset == 'train': 
            for i,f in enumerate(self.df['solutions'].iloc[indexes]):
                self.info[index*self.batch_size+i]=f
                y[i,] = np.array(list(map(int,list(f)))).reshape((81,1)) - 1
        if self.subset == 'train': return X, y
        else: return X

In [49]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

model.add(Flatten())
model.add(Dense(81*9))
model.add(Reshape((-1, 9)))
model.add(Activation('softmax'))

adam = keras.optimizers.adam(lr=.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [50]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 9, 9, 64)          640       
_________________________________________________________________
batch_normalization_3 (Batch (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 9, 9, 64)          36928     
_________________________________________________________________
batch_normalization_4 (Batch (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 9, 9, 128)         8320      
_________________________________________________________________
flatten_2 (Flatten)          (None, 10368)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 729)              

### Data Generators

In [51]:
train_idx = int(len(data)*0.95)
data = data.sample(frac=1).reset_index(drop=True)
training_generator = DataGenerator(data.iloc[:train_idx], subset = "train", batch_size=640)
validation_generator = DataGenerator(data.iloc[train_idx:], subset = "train",  batch_size=640)

In [52]:
training_generator.__getitem__(4)[0].shape

(640, 9, 9, 1)

### Callbacks


In [53]:
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
filepath1="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
filepath2 = "best_weights.hdf5"
checkpoint1 = ModelCheckpoint(filepath1, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
checkpoint2 = ModelCheckpoint(filepath2, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    verbose=1,
    min_lr=1e-6
)
callbacks_list = [checkpoint1,checkpoint2,reduce_lr]

In [54]:
history = model.fit_generator(training_generator, validation_data = validation_generator, epochs = 1, verbose=1,callbacks=callbacks_list )

Epoch 1/1
1484/1484 [==============================] - 125s 85ms/step - loss: 0.7142 - accuracy: 0.7115 - val_loss: 0.4081 - val_accuracy: 0.8133

Epoch 00001: val_accuracy improved from -inf to 0.81330, saving model to weights-improvement-01-0.81.hdf5

Epoch 00001: val_accuracy improved from -inf to 0.81330, saving model to best_weights.hdf5


## Performance

In [102]:
%%time
tmp = np.array([
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]).reshape((9,9,1))/9 - 0.5

# Reshape to include batch size of 1
tmp = np.expand_dims(tmp, axis=0)

# Now tmp should have a shape of (1, 9, 9, 1), suitable for the model
model_output = model.predict(tmp)

# Convert probabilities to choices (add 1 for Sudoku numbers)
sudoku_solution = np.argmax(model_output, axis=-1) + 1

# Reshape to a 9x9 grid
sudoku_solution = sudoku_solution.reshape((9, 9))

print(sudoku_solution)

[[5 3 1 6 7 8 9 9 8]
 [6 7 7 1 9 5 8 3 8]
 [1 9 8 3 3 2 5 6 2]
 [8 1 5 9 6 1 7 2 3]
 [4 2 6 8 5 3 9 5 1]
 [7 1 3 9 2 1 8 9 6]
 [9 6 9 7 5 7 2 8 4]
 [3 8 3 4 1 9 6 3 5]
 [3 5 5 2 8 6 6 7 9]]
CPU times: user 6.32 ms, sys: 146 µs, total: 6.46 ms
Wall time: 3.82 ms


## <span style="color:green;text-decoration:underline">Boosted BackTrack</span>


In [98]:
import time

def print_board(board):
    for row in board:
        print(" ".join(str(num) if num != 0 else '.' for num in row))

def find_empty_location(board):
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                return i, j
    return -1, -1

def is_safe(board, row, col, num):
    # Check if 'num' is not in the given row
    if num in board[row]:
        return False

    # Check if 'num' is not in the given column
    for i in range(9):
        if board[i][col] == num:
            return False

    # Check if 'num' is not in the given 3x3 box
    box_start_row, box_start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if board[i + box_start_row][j + box_start_col] == num:
                return False

    return True

def solve_sudoku_ml(board, predictions):
    row, col = find_empty_location(board)

    # If no empty space is left, puzzle is solved
    if row == -1 and col == -1:
        return True

    # Order the numbers to try based on model's predictions

    prob = predictions[row][col]
    numbers_to_try = np.argsort(prob)[::-1] + 1
    for num in numbers_to_try:
        if is_safe(board, row, col, num):
            board[row][col] = num

            if solve_sudoku_ml(board, predictions):
                return True

            # Backtrack
            board[row][col] = 0

    return False



In [99]:
%%time
# Example usage
puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]
tmp = np.array( puzzle).reshape((9,9,1))/9 - 0.5
# Reshape to include batch size of 1
tmp = np.expand_dims(tmp, axis=0)
predictions = model.predict(tmp)  
predictions = predictions.reshape((9, 9, 9))
if solve_sudoku_ml(puzzle, predictions):
    print_board(puzzle)
else:
    print("No solution exists")


5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9
CPU times: user 45.2 ms, sys: 2.03 ms, total: 47.2 ms
Wall time: 43.1 ms
